# Phase 5: Integrated Approach (监督+无监督混合方案)

## 目标

整合Phase 3的监督学习和Phase 4的无监督洞察，探索混合方案能否提升性能。

## 4种混合策略

1. **聚类标签作为特征** - 将聚类ID加入特征集
2. **异常分数作为特征** - 利用异常检测结果
3. **PCA降维后建模** - 在降维空间训练
4. **分群建模** - 为每个聚类训练独立模型

## 预期结果

- 至少1种方案性能 ≥ Phase 3最佳模型
- 理解混合方案的适用场景
- 选择最佳方案进入Phase 6

---

## 1. 环境准备

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ML库
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 自定义模块
from src import data_preprocessing as dp
from src import model_evaluation as me
from src import visualization as viz
from config import get_default_config

# 配置
plt.style.use('seaborn-v0_8-darkgrid')
config = get_default_config()
RANDOM_STATE = config.RANDOM_STATE

print("✅ 环境准备完成!")
print(f"随机种子: {RANDOM_STATE}")

## 2. 回顾Phase 3和Phase 4

加载前置Phase的结果，建立性能基线。

In [ ]:
# 加载数据（重新预处理，保持一致性）
df = sns.load_dataset('titanic')
target_col = 'survived'

# 快速预处理（与Phase 3一致）
X, y = dp.quick_preprocess(
    df, 
    target=target_col,
    handle_missing='simple',
    encode_categorical='onehot',
    scale_features=True
)

# 数据划分（与Phase 3一致）
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=RANDOM_STATE, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=RANDOM_STATE, stratify=y_temp
)

print(f"训练集: {X_train.shape}")
print(f"验证集: {X_val.shape}")
print(f"测试集: {X_test.shape}")

In [ ]:
# Phase 3 Baseline: 训练一个标准的随机森林作为对比基准
print("Phase 3 Baseline (标准监督学习):")
print("=" * 50)

baseline_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=RANDOM_STATE
)
baseline_model.fit(X_train, y_train)

baseline_val_score = baseline_model.score(X_val, y_val)
baseline_test_score = baseline_model.score(X_test, y_test)

print(f"验证集准确率: {baseline_val_score:.4f}")
print(f"测试集准确率: {baseline_test_score:.4f}")
print("\n这是Phase 5要超越的基准!")

## 3. Phase 4回顾: 准备无监督特征

由于Phase 4可能尚未运行或保存特征，这里我们快速生成无监督特征。

In [ ]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.ensemble import IsolationForest

print("生成Phase 4的无监督特征...")
print("=" * 50)

# 1. K-Means聚类（3个簇）
kmeans = KMeans(n_clusters=3, random_state=RANDOM_STATE)
cluster_labels_train = kmeans.fit_predict(X_train)
cluster_labels_val = kmeans.predict(X_val)
cluster_labels_test = kmeans.predict(X_test)

print(f"✓ K-Means聚类完成: {len(np.unique(cluster_labels_train))} 个簇")

# 2. Isolation Forest异常检测
iso_forest = IsolationForest(contamination=0.1, random_state=RANDOM_STATE)
iso_forest.fit(X_train)

anomaly_scores_train = iso_forest.score_samples(X_train)
anomaly_scores_val = iso_forest.score_samples(X_val)
anomaly_scores_test = iso_forest.score_samples(X_test)

print(f"✓ 异常检测完成: 异常分数范围 [{anomaly_scores_train.min():.2f}, {anomaly_scores_train.max():.2f}]")

# 3. PCA降维（保留95%方差）
pca = PCA(n_components=0.95, random_state=RANDOM_STATE)
X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

print(f"✓ PCA降维完成: {X_train.shape[1]} → {X_train_pca.shape[1]} 维")
print(f"  保留方差比: {pca.explained_variance_ratio_.sum():.2%}")

print("\n✅ 无监督特征准备完毕!")

---

## 4. 方案1: 聚类标签作为特征

将K-Means聚类的簇ID作为新特征（One-Hot编码）加入训练。

In [ ]:
print("方案1: 聚类标签作为特征")
print("=" * 60)

# One-Hot编码聚类标签
cluster_df_train = pd.get_dummies(cluster_labels_train, prefix='cluster')
cluster_df_val = pd.get_dummies(cluster_labels_val, prefix='cluster')
cluster_df_test = pd.get_dummies(cluster_labels_test, prefix='cluster')

# 合并到原特征集
X_train_cluster = np.hstack([X_train, cluster_df_train.values])
X_val_cluster = np.hstack([X_val, cluster_df_val.values])
X_test_cluster = np.hstack([X_test, cluster_df_test.values])

print(f"特征数量: {X_train.shape[1]} → {X_train_cluster.shape[1]} (+{cluster_df_train.shape[1]} 个聚类特征)")

# 训练模型
model_cluster = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=RANDOM_STATE
)
model_cluster.fit(X_train_cluster, y_train)

# 评估
val_score_cluster = model_cluster.score(X_val_cluster, y_val)
test_score_cluster = model_cluster.score(X_test_cluster, y_test)

print(f"\n验证集准确率: {val_score_cluster:.4f} (Baseline: {baseline_val_score:.4f})")
print(f"测试集准确率: {test_score_cluster:.4f} (Baseline: {baseline_test_score:.4f})")

improvement_1 = (val_score_cluster - baseline_val_score) / baseline_val_score * 100
print(f"性能变化: {improvement_1:+.2f}%")

---

## 5. 方案2: 异常分数作为特征

将Isolation Forest的异常分数标准化后作为新特征。

In [ ]:
print("方案2: 异常分数作为特征")
print("=" * 60)

# 标准化异常分数
scaler_anomaly = StandardScaler()
anomaly_scaled_train = scaler_anomaly.fit_transform(anomaly_scores_train.reshape(-1, 1))
anomaly_scaled_val = scaler_anomaly.transform(anomaly_scores_val.reshape(-1, 1))
anomaly_scaled_test = scaler_anomaly.transform(anomaly_scores_test.reshape(-1, 1))

# 合并到原特征集
X_train_anomaly = np.hstack([X_train, anomaly_scaled_train])
X_val_anomaly = np.hstack([X_val, anomaly_scaled_val])
X_test_anomaly = np.hstack([X_test, anomaly_scaled_test])

print(f"特征数量: {X_train.shape[1]} → {X_train_anomaly.shape[1]} (+1 个异常分数特征)")

# 训练模型
model_anomaly = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=RANDOM_STATE
)
model_anomaly.fit(X_train_anomaly, y_train)

# 评估
val_score_anomaly = model_anomaly.score(X_val_anomaly, y_val)
test_score_anomaly = model_anomaly.score(X_test_anomaly, y_test)

print(f"\n验证集准确率: {val_score_anomaly:.4f} (Baseline: {baseline_val_score:.4f})")
print(f"测试集准确率: {test_score_anomaly:.4f} (Baseline: {baseline_test_score:.4f})")

improvement_2 = (val_score_anomaly - baseline_val_score) / baseline_val_score * 100
print(f"性能变化: {improvement_2:+.2f}%")

---

## 6. 方案3: PCA降维后建模

在降维空间训练模型，减少特征数量，可能降低过拟合风险。

In [ ]:
print("方案3: PCA降维后建模")
print("=" * 60)

print(f"特征数量: {X_train.shape[1]} → {X_train_pca.shape[1]} (降维{X_train.shape[1] - X_train_pca.shape[1]}维)")

# 训练模型
import time
start_time = time.time()

model_pca = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=RANDOM_STATE
)
model_pca.fit(X_train_pca, y_train)

train_time_pca = time.time() - start_time

# 评估
val_score_pca = model_pca.score(X_val_pca, y_val)
test_score_pca = model_pca.score(X_test_pca, y_test)

print(f"\n验证集准确率: {val_score_pca:.4f} (Baseline: {baseline_val_score:.4f})")
print(f"测试集准确率: {test_score_pca:.4f} (Baseline: {baseline_test_score:.4f})")
print(f"训练时间: {train_time_pca:.3f}秒")

improvement_3 = (val_score_pca - baseline_val_score) / baseline_val_score * 100
print(f"性能变化: {improvement_3:+.2f}%")

---

## 7. 方案4: 分群建模

为每个聚类簇训练独立模型，预测时根据样本所属簇选择对应模型。

In [ ]:
print("方案4: 分群建模")
print("=" * 60)

n_clusters = len(np.unique(cluster_labels_train))
print(f"训练{n_clusters}个簇的独立模型...\n")

# 为每个簇训练模型
cluster_models = {}
cluster_sizes = {}

for cluster_id in range(n_clusters):
    # 获取该簇的数据
    mask_train = cluster_labels_train == cluster_id
    X_cluster = X_train[mask_train]
    y_cluster = y_train[mask_train]
    
    cluster_sizes[cluster_id] = len(X_cluster)
    
    # 训练模型
    model = RandomForestClassifier(
        n_estimators=50,  # 减少树数量（簇内样本较少）
        max_depth=8,
        random_state=RANDOM_STATE
    )
    model.fit(X_cluster, y_cluster)
    cluster_models[cluster_id] = model
    
    print(f"  簇{cluster_id}: {len(X_cluster)} 个样本")

print("\n✓ 所有簇的模型训练完成")

In [ ]:
# 预测函数
def predict_by_cluster(X, cluster_labels, cluster_models):
    """根据聚类标签选择对应模型进行预测"""
    predictions = np.zeros(len(X))
    
    for cluster_id, model in cluster_models.items():
        mask = cluster_labels == cluster_id
        if mask.sum() > 0:
            predictions[mask] = model.predict(X[mask])
    
    return predictions

# 验证集预测
y_val_pred = predict_by_cluster(X_val, cluster_labels_val, cluster_models)
val_score_cluster_models = accuracy_score(y_val, y_val_pred)

# 测试集预测
y_test_pred = predict_by_cluster(X_test, cluster_labels_test, cluster_models)
test_score_cluster_models = accuracy_score(y_test, y_test_pred)

print(f"验证集准确率: {val_score_cluster_models:.4f} (Baseline: {baseline_val_score:.4f})")
print(f"测试集准确率: {test_score_cluster_models:.4f} (Baseline: {baseline_test_score:.4f})")

improvement_4 = (val_score_cluster_models - baseline_val_score) / baseline_val_score * 100
print(f"性能变化: {improvement_4:+.2f}%")

---

## 8. 方案对比与最佳选择

In [ ]:
# 汇总所有方案的结果
comparison_df = pd.DataFrame({
    '方案': [
        'Baseline (Phase 3)',
        '方案1: 聚类标签特征',
        '方案2: 异常分数特征',
        '方案3: PCA降维',
        '方案4: 分群建模'
    ],
    '验证集准确率': [
        baseline_val_score,
        val_score_cluster,
        val_score_anomaly,
        val_score_pca,
        val_score_cluster_models
    ],
    '测试集准确率': [
        baseline_test_score,
        test_score_cluster,
        test_score_anomaly,
        test_score_pca,
        test_score_cluster_models
    ],
    '性能变化%': [
        0.0,
        improvement_1,
        improvement_2,
        improvement_3,
        improvement_4
    ],
    '特征数': [
        X_train.shape[1],
        X_train_cluster.shape[1],
        X_train_anomaly.shape[1],
        X_train_pca.shape[1],
        X_train.shape[1]
    ]
})

# 按验证集准确率排序
comparison_df = comparison_df.sort_values('验证集准确率', ascending=False)

print("\n" + "=" * 80)
print("Phase 5 混合方案对比".center(80))
print("=" * 80)
print(comparison_df.to_string(index=False))
print("=" * 80)

In [ ]:
# 可视化对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左图: 验证集准确率对比
ax1 = axes[0]
bars1 = ax1.barh(comparison_df['方案'], comparison_df['验证集准确率'])
bars1[0].set_color('gray')  # Baseline用灰色
ax1.set_xlabel('验证集准确率', fontsize=12)
ax1.set_title('验证集性能对比', fontsize=14, fontweight='bold')
ax1.axvline(baseline_val_score, color='red', linestyle='--', label='Baseline')
ax1.legend()

# 右图: 性能变化百分比
ax2 = axes[1]
colors = ['gray' if x == 0 else 'green' if x > 0 else 'red' 
          for x in comparison_df['性能变化%']]
bars2 = ax2.barh(comparison_df['方案'], comparison_df['性能变化%'], color=colors)
ax2.set_xlabel('性能变化 (%)', fontsize=12)
ax2.set_title('相对Baseline的性能变化', fontsize=14, fontweight='bold')
ax2.axvline(0, color='black', linestyle='-', linewidth=0.8)

plt.tight_layout()
plt.show()

print("\n图例: 绿色=性能提升, 红色=性能下降, 灰色=Baseline")

In [ ]:
# 选择最佳方案
best_method = comparison_df.iloc[0]

print("\n" + "🏆 " * 20)
print(f"\n最佳方案: {best_method['方案']}")
print(f"验证集准确率: {best_method['验证集准确率']:.4f}")
print(f"测试集准确率: {best_method['测试集准确率']:.4f}")
print(f"性能变化: {best_method['性能变化%']:+.2f}%")

if best_method['性能变化%'] > 0:
    print(f"\n✅ 混合方案成功提升了性能!")
elif best_method['性能变化%'] == 0:
    print(f"\n✓ 混合方案与Baseline持平（这也是正常结果）")
else:
    print(f"\n⚠️  混合方案未能提升性能（可能数据集较简单，Baseline已足够）")

print("\n" + "🏆 " * 20)

---

## 9. Phase 5 总结

In [ ]:
print("=" * 80)
print("Phase 5 混合方案总结".center(80))
print("=" * 80)

print("\n1. 实验了4种混合方案:")
print("   ✓ 方案1: 聚类标签特征 - 增加簇归属信息")
print("   ✓ 方案2: 异常分数特征 - 利用异常检测结果")
print("   ✓ 方案3: PCA降维建模 - 减少特征维度")
print("   ✓ 方案4: 分群建模 - 为每个簇独立建模")

print("\n2. 性能对比:")
best_improvement = comparison_df[comparison_df['方案'] != 'Baseline (Phase 3)']['性能变化%'].max()
worst_improvement = comparison_df[comparison_df['方案'] != 'Baseline (Phase 3)']['性能变化%'].min()
print(f"   最佳提升: {best_improvement:+.2f}%")
print(f"   最差变化: {worst_improvement:+.2f}%")

print("\n3. 关键发现:")
if best_improvement > 1:
    print("   ✅ 混合方案显著提升了性能")
    print("   → 建议: 将最佳方案用于Phase 6最终模型")
elif best_improvement > 0:
    print("   ✓ 混合方案有轻微提升")
    print("   → 建议: 可考虑采用，但提升有限")
else:
    print("   ⚠️  混合方案未能改善性能")
    print("   → 原因: 可能数据集较简单，Baseline已接近最优")
    print("   → 建议: Phase 6继续使用Phase 3的最佳模型")

print("\n4. 适用场景总结:")
print("   - 聚类标签特征: 数据有明显子群体时")
print("   - 异常分数特征: 异常样本有特殊模式时")
print("   - PCA降维: 特征高维且有冗余时")
print("   - 分群建模: 不同子群体规律差异大时")

print("\n5. 下一步:")
print(f"   → 进入Phase 6，使用 '{best_method['方案']}' 构建最终Pipeline")

print("\n" + "=" * 80)
print("✅ Phase 5 完成! 下一步: phase6_final_solution.ipynb")
print("=" * 80)

## 参考资料

- **Phase 3结果**: `phase3_supervised_solution.ipynb` - 监督学习最佳模型
- **Phase 4结果**: `phase4_unsupervised_insights.ipynb` - 无监督学习洞察
- **工作流指南**: `../ML_WORKFLOW_GUIDE.md`

---

**下一步**: [Phase 6: 最终方案](phase6_final_solution.ipynb)